# Bouton density

- Calculate bouton density
- Validate bouton density

<img src="https://raw.githubusercontent.com/antonelepfl/hbp_school/master/bouton_density.png">

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bluepy.v2 import Cell, Circuit
from connectome_tools.dataset import read_bouton_density

In [ ]:
circuit = Circuit('circuit/CircuitConfig')
mtypes = sorted(circuit.cells.mtypes)

In [ ]:
SAMPLE_SIZE = 10

In [ ]:
bouton_densities = {}
for mtype in mtypes:
    group = {Cell.MTYPE: mtype, Cell.REGION: '@mc2.*'}
    sample = circuit.stats.sample_bouton_density(SAMPLE_SIZE, group, synapses_per_bouton=1.2)
    bouton_densities[mtype] = {
        'mean': np.mean(sample),
        'std': np.std(sample)
    }
bouton_densities = pd.DataFrame(bouton_densities).transpose()
bouton_densities.head()

In [ ]:
ax = bouton_densities.plot.bar(yerr='std', title='Bouton density', legend=False)
ax.set_xlabel('mtype')
ax.set_ylabel('density (um^-1)')
ax.set_xticklabels(labels=bouton_densities.index, rotation=45);

In [ ]:
bio_path = 'circuit/bioname/bouton_density_20180125.tsv'

In [ ]:
data = read_bouton_density(bio_path).rename(columns={
    'mean': 'bio_mean',
    'std': 'bio_std'
})
data.drop(0, inplace=True) 
data.drop(['size', 'sample'], axis=1, inplace=True) 
data.head()

In [ ]:
selected = data['mtype'].values
data['mod_mean'] = bouton_densities['mean'][selected].values
data['mod_std'] = bouton_densities['std'][selected].values
data.dropna(inplace=True)
data.head()

In [ ]:
x = data['mod_mean'].values
y = data['bio_mean'].values
# remove nan value from the array
l = np.linspace(0, max(x[~np.isnan(x)].max(), y.max()), 50)
#plt.plot(x, y, 'o')
plt.errorbar(x, y, xerr=data['mod_std'].values, yerr=data['bio_std'].values, fmt='o', ecolor='g', capthick=2)
plt.plot(l, l, 'k--')
plt.xlabel('Model (um^-1)')
plt.ylabel('Experiment (um^-1)')
plt.title('Bouton density')